In [0]:
from pyspark.sql.functions import *

In [0]:
df_policy_effect = spark.read.format("delta").load("/mnt/gold/policy_effect_on_covid")
df_mobility_effect = spark.read.format("delta").load("/mnt/gold/mobility_effects")

In [0]:
df_mobility_effect_2 = df_mobility_effect.select(
    "location", "year", "month",
    "avg_retail", "avg_grocery", "avg_workplace", "avg_home",
    "workplace_reduction_rank", "home_stay_rank"
)
df_joined = df_policy_effect.join(
    df_mobility_effect_2,
    on=["location", "year", "month"],
    how="inner"
)

In [0]:
df_joined_2 = df_joined.withColumn(
    "combined_policy_mobility_effective",
    when(
        (col("monthly_policy_strictness_rank") >= 10) & (col("home_stay_rank") <= 5),
        1
    ).otherwise(0)
)

In [0]:
df_joined_2.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save("/mnt/gold/combined_policy_mobility_effects")